# Paradox Home Assignment
Aviv Harazi

The assignment splits to two parts:
1. Generation of fictional GenAI Summer Camp 
2. Building LLM Assistant Agent 


### Generation of fictional GenAI Summer Camp 
First, I made a template for the program and what it should have.
This template has a few keys: campName, description, offerings, values, policies, location, dates, pricing, contact, highlights, conclusion.

Once I had the template, the process of creating content happens in four steps. Each step works on certain parts from the template to get more words.
All these four steps run together and then come together at the end.
The final result is in the form of a dictionary, which is easier to work with.
I also designed an enhancement function with the aim of generating more words and enriching the text.
The final output is dictionary filled with correct information.

### Building LLM Assistant Agent 
I employed Langchain to establish an Agent that utilizes custom tools, granting us the capability to carry out actions. The Agent determines which tool to employ based on the input and tool descriptions available.

I developed a few tools:

* Register Application Tool: When a parent expresses interest in registering, this tool activates the registration process. It collects parent names, phone numbers, emails, and child ages, and validates the age against camp details. Each application is saved in dictionary format.

* Answer Pricing or Age-Related Questions: For questions related to pricing or age, the Agent employs a GPT model with pertinent pricing and age information derived from camp details.

* Answer Policies and Location Questions: In response to queries regarding policies, location or dates, the Agent employs a GPT model with relevant details from camp information.

* Answer General Questions: If queries are general or require more information, the Agent employs a GPT model using pertinent information from camp details.

* Finish Process: If there are no further actions required, the process is concluded.


## How to run? 
* Open the notebook via jupyter notebook / google collab
* Install the required libraries 
* Set the API key in openai.api_key parameter 
* use agent.run() examples to add and check more scenarios.

In [22]:
# required libraries if not installed
!pip install openai==0.27.2
!pip install numpy
!pip install langchain==0.0.270

  Using cached langchain-0.0.270-py3-none-any.whl (1.5 MB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.27
    Uninstalling langchain-0.0.27:
      Successfully uninstalled langchain-0.0.27


In [1]:
# imports
import openai
import os
import numpy as np
import ast
import time
from multiprocessing.pool import ThreadPool
import multiprocessing
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain import OpenAI
from langchain.agents import initialize_agent
openai.api_key = 'API_KEY_HERE'

# Generation of fictional GenAI Summer Camp

In [2]:
## main prompts in dict
db_prompts = {
    "generate_camp_stage1": """Create the introduction and offerings section (~1500 words) for a fictional summer camp called "GenAI Summer Camp." Craft an engaging description that introduces the camp's concept and its various offerings. The offerings should cover a range of activities such as AI workshops, creative coding, robotics exploration, virtual reality adventures, ethics and AI discussions, innovative app development, and tech and nature fusion. Aim for clarity, excitement, and creativity in your description. Provide enough detail to make each offering enticing while maintaining an engaging tone.
Put this message in the following JSON structure:
{
  "campName": "..",
  "description": "..",
  "offerings": [
    {
      "title": "..",
      "description": ".."
    },
    {
      "title": "..",
      "description": ".."
    },
    ...
  ]
}
""",
    
    "generate_camp_stage2": """Create the section that outlines the camp's values, policies, and location (~1600 words). Describe the core values such as curiosity, collaboration, responsibility, and inclusivity. Provide succinct explanations for each policy, including safety first, respectful behavior, zero tolerance for bullying, and parental engagement. Offer a brief description of the camp's location, emphasizing its natural beauty and suitability for learning.
Put this message in the following JSON structure:
{
  "values": [
    ""
  ],
  "policies": [
    {
      "title": "..",
      "description": "..",
      "importance": ".."
    },
    {
      "title": "..",
      "description": "..",
      "importance": ".."
    },
    {
      "title": "..",
      "description": "..",
      "importance": ".."
    },
    ...
  ],
  "location": {
    "name": "..",
    "description": ".."
  }
}
""",
    "generate_camp_stage3": """Craft the section that covers camp dates, pricing, and contact details (~1500 words). Provide the start and end dates of the camp along with its duration. Outline the pricing structure for different age ranges, including early bird and regular prices. Include the camp's contact information, including email and phone number.
Put this message in the following JSON structure:
{
  "dates": {
    "start": "..",
    "end": "..",
    "duration": ".."
  },
  "pricing": [
    {
      "ageRange": "..",
      "earlyBirdPrice": .. ,
      "regularPrice": ..
    },
    {
      "ageRange": "..",
      "earlyBirdPrice":.. ,
      "regularPrice": .. 
    },
    {
      "ageRange": "..",
      "earlyBirdPrice":.. ,
      "regularPrice": .. 
    },
  ],
  "contact": {
    "email": "..",
    "phone": ".."
  }
}
""",
    
    "generate_camp_stage4" : """Create the section that highlights the key features of the camp and provides a concluding statement (~1200 words). List the highlights, such as hands-on AI workshops, innovative tech and arts fusion, thrilling robotics challenges, immersive VR experiences, ethics discussions, tech and nature explorations, and emphasis on safety, respect, and inclusivity. Wrap up the description with an encouraging closing statement inviting potential campers to join the exciting journey.
Put this message in the following JSON structure:
{
  "highlights": [
    "..",
    "..",
    ...
  ],
  "conclusion" : ".."
}
""",
    "enhance_prompt": "Your job is to get JSON format input and double the size of word count for each value. the text need to be richer and in more detail. raise the number of total words. return the your final output as JSON without word count key."
}

In [3]:
# functions to generate the GenAI Summer Camp details
def send_request(myprompt,temperature=0.95,max_tokens=1200,model_name="text-davinci-003",parse=True,enhance=True):
    """
    Sends a completion request to OpenAI's API using the specified prompt and arguments.

    Args:
        myprompt (str): The prompt for generating the completion.
        temperature (float, optional): Sampling temperature for randomness. Defaults to 0.95.
        max_tokens (int, optional): The maximum number of tokens in the generated output. Defaults to 1200.
        model_name (str, optional): The name of the OpenAI model to use. Defaults to "text-davinci-003".
        parse (bool, optional): Whether to parse the generated response as a Python literal using ast.literal_eval().
                               Defaults to True.
        enhance (bool, optional): Whether to enhance the response using the `enhance_request` function if the length of
                                 the response is less than half of max_tokens. Defaults to True.

    Returns:
        str: The generated response text.
    """
    res = None
    try:
        response = openai.Completion.create(
              model=model_name,
              prompt=myprompt,
              temperature=temperature,
              max_tokens=max_tokens,
            )
        
        res = response["choices"][0]["text"]
        if parse:
            res = ast.literal_eval(res)
        
        if enhance and len(str(res).split(" ")) < max_tokens/2:
            res = enhance_request(str(res))
            
        return res
            
    except Exception as e:
        print(e)
        return send_request(myprompt,temperature,max_tokens,model_name,parse,enhance)
    

def enhance_request(req,model_name="gpt-3.5-turbo"):
    """
    Enhances a given request using a ChatCompletion model.

    Args:
        req (str): The request to be enhanced.
        model_name (str, optional): The name of the OpenAI model to use. Defaults to "gpt-3.5-turbo".

    Returns:
        str: The enhanced response text.
    """
    try:
        response = openai.ChatCompletion.create(
              model=model_name,
              messages=[
                      {"role": "system", "content": db_prompts['enhance_prompt']},
                      {"role": "user", "content": req}
                    ],
              temperature=0.95,

            )

        text = response["choices"][0]["message"]["content"]
    

        return ast.literal_eval(text)
    except Exception as e:
        print(e)
        return enhance_request(req,model_name)
    
        
def process_request(params):
    """
    Processes a request using the send_request function (wrapper for multiprocessing usage)

    Args:
        params (tuple): A tuple containing the prompt, temperature, and max_tokens.

    Returns:
        str: The generated response text.
    """
    prompt, temperature, max_tokens = params
    return send_request(prompt, temperature, max_tokens)

    
def generate_camp():
    """
    Generates GenAI summer camp details in parallel

    Returns:
        dict: A dictionary containing generated response
    """
    final_res = {}
    # list of prompts with specific arguments for each
    all_prompts = [(db_prompts['generate_camp_stage1'],0.5,3700),(db_prompts['generate_camp_stage2'], 0.5,3700),(db_prompts['generate_camp_stage3'],0.5,3700),(db_prompts['generate_camp_stage4'],0.5,3500)]
    
    # pool of threads
    with ThreadPool(processes=len(all_prompts)) as pool:
        results = pool.map(process_request, all_prompts)
        
    
    # update the base dict
    [final_res.update(res) for res in results]
    
    return final_res
    

In [4]:
# generate camp details
camp_details = generate_camp()

In [5]:
camp_details

{'campName': 'GenAI Summer Camp',
 'description': "Welcome to GenAI Summer Camp – come join us on an exciting and educational adventure as we explore the endless possibilities of modern technology! We offer a wide range of engaging and thought-provoking activities that combine the thrill of creative coding, immersive robotics exploration, mind-blowing virtual reality experiences, and innovative app development with in-depth discussions on the ethical implications of artificial intelligence. At GenAI Summer Camp, you'll not only learn new skills and discover cutting-edge technologies, but you'll also make lifelong friendships and create unforgettable memories through our groundbreaking programs. Get ready for an unforgettable summer filled with groundbreaking advancements in AI, enlightening conversations, and the opportunity to push the boundaries of what you once believed to be possible. Don't miss out - sign up now and prepare to embark on an extraordinary journey into the world of A

# Building LLM Assistant Agent

In [6]:
## Register Application form
application_form = {
    'parents_full_names' : "",
    'parents_phone_number' : "",
    'email' : "",
    'kid_age': ""  
}
all_applications = []

In [7]:
# tools functions
def answer_pricing_age_questions(text):
    """
    Generates responses to pricing and age-related questions.

    Args:
        text (str): The question text.

    Returns:
        str: The generated response.
    """
    global camp_details
    myprompt = f"""Answer the question based on the following GenAI Summer camp details in dictionary: age express as range, early price as number and regular price as number:
    {camp_details['pricing']}
    
    Question:
    {text}
    """
    res = send_request(myprompt,temperature=0,max_tokens=None,parse=False,enhance=False)
    
    return res


def answer_general_questions(text):
    """
    Generates responses to general questions about the camp.

    Args:
        text (str): The question text.

    Returns:
        str: The generated response.
    """
    global camp_details
    temp_dict = {key: camp_details[key] for key in ['description','offerings','highlights','values','conclusion']}
    myprompt = f"""Answer the question based on the following GenAI Summer camp details in dictionary:
    {temp_dict}
    
    Question:
    {text}
    """
    res = send_request(myprompt,temperature=0,max_tokens=None,parse=False,enhance=False)
    
    return res

def answer_policies_location_questions(text):
    """
    Generates responses to questions about policies and location.

    Args:
        text (str): The question text.

    Returns:
        str: The generated response.
    """
    global camp_details
    temp_dict = {key: camp_details[key] for key in ['policies','location','dates']}
    myprompt = f"""Answer the question based on the following GenAI Summer camp details in dictionary:
    {temp_dict}
    
    Question:
    {text}
    """
    res = send_request(myprompt,temperature=0,max_tokens=None,parse=False,enhance=False)
    
    return res


def start_application(text):
    """
    Initiates the registration process for the GenAI summer camp.

    Args:
        text (str): The initial text.

    Returns:
        dict or str: The application form or an end message.
    """
    global camp_details,all_applications
    print("___________________")
    print("Thank you so much for signing up for our summer camp! We're thrilled to have you on board. Get ready for a fantastic time of fun and learning.")
    
    parents_name = input("Could you provide the complete names of the parents?")
    parents_phone = input("Please provide the phone numbers of your parents ")
    email = input("Almost done, What is your email?")
    kid_age = input("What is your kid age?")
    
    # validate kid age
    myprompt = f"""Verify the given age with policy of age range represented as dictionary, if the given age exists in any program answer valid or not valid.
    Age: {kid_age}
    Age policy: {camp_details['pricing']}
    """
    age_validation = send_request(myprompt,temperature=0,max_tokens=None,parse=False,enhance=False)
    #print("Age validation: ", age_validation)
    if 'not' in str(age_validation).lower():
        print("We inform the individual that their age is not considered valid, therefore, we do not have an appropriate program available for them right now.")
        return "Provided Age is not valid. END OF FLOW"
    
    final_app_form = application_form.copy()
    final_app_form['parents_full_names'] = parents_name
    final_app_form['parents_phone_number'] = parents_phone
    final_app_form['email'] = email
    final_app_form['kid_age'] = kid_age
    
    all_applications.append(final_app_form)
    
    return "Registration has been successfully completed, You will receive an email with more details shortly. END OF FLOW"

def finish_process(text):
    """
    Concludes the summer camp application process.

    Args:
        text (str): The closing text.

    Returns:
        str: The end message.
    """
    print("Thank you for expressing your interest in joining the GenAI summer camp. We will be in contact with you shortly!")
    return "End of flow"

In [8]:
# declaring llm and tools
llm = OpenAI(temperature=0,openai_api_key=openai.api_key)
tools = [
    Tool.from_function(
        func=answer_general_questions,
        name="Answer general camp questions",
        description="useful when parents have general questions on GenAI summer camp or want more details"
    ),
    Tool.from_function(
        func=answer_pricing_age_questions,
        name="Answer age or pricing camp questions",
        description="useful when parents have age or price questions on GenAI summer camp"
    ),
    Tool.from_function(
        func=answer_policies_location_questions,
        name="Answer policies or location camp questions",
        description="useful when parents have policies questions, location questions or dates questions on GenAI summer camp"
    ),
    Tool.from_function(
        func=start_application,
        name="Register application to GenAI summer camp",
        description="useful when kid or parents want to register to GenAI summer camp"
    ),
    Tool.from_function(
        func=finish_process,
        name="After registeration to GenAI summer camp",
        description="useful when parent finish the registeration process"
    ),
]

In [9]:
# setting up the agent
agent = initialize_agent(
    tools,
   llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# Agent Test Scenarios

In [10]:
agent.run("explain me what my child will do in the summer camp?")



> Entering new AgentExecutor chain...
 I need to provide parents with general camp details
Action: Answer general camp questions
Action Input: What my child will do in the summer camp?
Observation: 
Your child will have the opportunity to engage in interactive and dynamic AI workshops,
Thought: I now know the final answer
Final Answer: Your child will have the opportunity to engage in interactive and dynamic AI workshops at our GenAI summer camp.

> Finished chain.


'Your child will have the opportunity to engage in interactive and dynamic AI workshops at our GenAI summer camp.'

In [11]:
agent.run("What is the price?")



> Entering new AgentExecutor chain...
 I should answer the question with the price information.
Action: Answer age or pricing camp questions
Action Input: Price
Observation: 
Answer: The early bird price for 5-7 years old is 700,
Thought: I now know the final answer
Final Answer: The early bird price for 5-7 years old is 700.

> Finished chain.


'The early bird price for 5-7 years old is 700.'

In [12]:
agent.run("What is the price for age 12?")



> Entering new AgentExecutor chain...
 I need to look up the price based on age.
Action: Answer age or pricing camp questions
Action Input: I want to know the price for age 12
Observation: 
The price for age 12 is 1400 (early Bird Price).
Thought: I now know the final answer
Final Answer: The price for age 12 is 1400 (early Bird Price).

> Finished chain.


'The price for age 12 is 1400 (early Bird Price).'

In [13]:
agent.run("What is the price for age 8?")



> Entering new AgentExecutor chain...
 This is an age and pricing question.
Action: Answer age or pricing camp questions
Action Input: Age 8
Observation: 
The early bird price for 8-11 years old is 1000.
Thought: I now know the final answer.
Final Answer: The early bird price for 8-11 years old is 1000.

> Finished chain.


'The early bird price for 8-11 years old is 1000.'

In [14]:
agent.run("The summer camp is safe?")



> Entering new AgentExecutor chain...
 I need to provide reassurance about the camp
Action: Answer general camp questions
Action Input: Information on camp safety
Observation: 
Answer: Safety is our top priority, and we provide a nurturing and supportive
Thought: environment for all campers. We also have a team of trained professionals supporting our campers to ensure their safety, health and wellbeing at all times.
Final Answer: Safety is our top priority, and we provide a nurturing and supportive environment for all campers. We also have a team of trained professionals supporting our campers to ensure their safety, health and wellbeing at all times.

> Finished chain.


'Safety is our top priority, and we provide a nurturing and supportive environment for all campers. We also have a team of trained professionals supporting our campers to ensure their safety, health and wellbeing at all times.'

In [15]:
agent.run("Can I have more details?")



> Entering new AgentExecutor chain...
 This question is general and about details that are not related to age, pricing, policies, locations or dates.
Answer general camp questions is the action to take.
Action: Answer general camp questions
Action Input: None
Observation: 
GenAI Summer Camp is an exciting and educational adventure that offers a range of
Thought: meaningful activities. 
Action: None
Final Answer: GenAI Summer Camp is an exciting and educational adventure that offers a range of meaningful activities.

> Finished chain.


'GenAI Summer Camp is an exciting and educational adventure that offers a range of meaningful activities.'

In [16]:
# failing due age validation
agent.run("I want to register")



> Entering new AgentExecutor chain...
 How can I guide them through the registration process?
Action: Register application to GenAI summer camp
Action Input: Provide information on the registration form as needed.
Action: After registeration to GenAI summer camp
Action Input: Provide the confirmation of the registration___________________
Thank you so much for signing up for our summer camp! We're thrilled to have you on board. Get ready for a fantastic time of fun and learning.
Could you provide the complete names of the parents?das asd 
Please provide the phone numbers of your parents dasd
Almost done, What is your email?dsadas
What is your kid age?40
We inform the individual that their age is not considered valid, therefore, we do not have an appropriate program available for them right now.

Observation: Provided Age is not valid. END OF FLOW
Thought: I should answer what age works
Final Answer: The GenAI summer camp is recommended for kids between the ages of 7 and 11.

> Finish

'The GenAI summer camp is recommended for kids between the ages of 7 and 11.'

In [17]:
agent.run("I want to register")



> Entering new AgentExecutor chain...
 I need to understand what's needed for registration
Action: Register application to GenAI summer camp
Action Input: Provide contact info, age, and payment info___________________
Thank you so much for signing up for our summer camp! We're thrilled to have you on board. Get ready for a fantastic time of fun and learning.
Could you provide the complete names of the parents?das
Please provide the phone numbers of your parents a
Almost done, What is your email?a
What is your kid age?8

Observation: Registration has been successfully completed, You will receive an email with more details shortly. END OF FLOW
Thought: I now know the final answer
Final Answer: You have successfully registered for the GenAI summer camp. You will receive an email with more details shortly.

> Finished chain.


'You have successfully registered for the GenAI summer camp. You will receive an email with more details shortly.'

In [18]:
agent.run("What my child would benefit from joining the GenAI camp?")



> Entering new AgentExecutor chain...
 I should determine what type of question this is.
Action: Answer general camp questions
Action Input: What my child would benefit from joining the GenAI camp?
Observation: 
Your child would benefit from joining GenAI Summer Camp by gaining hands-on
Thought: I now know the final answer
Final Answer: Your child would benefit from joining GenAI Summer Camp by gaining hands-on experience with AI development, working on projects with industry professionals, and more.

> Finished chain.


'Your child would benefit from joining GenAI Summer Camp by gaining hands-on experience with AI development, working on projects with industry professionals, and more.'

# Open Questions

1.

* I would conduct further exploration of the hyperparameters API and deliberately select the appropriate ones for each scenario (temperature, top_p).

* Enhancing the handling of exceptions is important. There might be instances where the response isn't in JSON format, leading to parsing errors. While my current approach involves resending the data in case of an exception, alternative strategies could be considered.

* I would explore strategies to maximize the number of words in each camp detail.

* Tailoring the choice of models to specific tasks.

* Consider summarizing summer camp details in certain situations.

* Optimize memory usage within the agent.

* Addressing the challenges posed by a larger context window is vital.

* Create and explore prompts in more stable way such as prompt template object from langchain.

* Create this application as a service outside jupyter notebook.

* Dealing with End Of Flow scenarios better.


2.

* Evaluate the model's performance across different usage scenarios, including tasks such as addressing registration-related queries and providing detailed information about camp activities. This assessment will help identify strengths and weaknesses in the model's response generation for various prompts.

* Construct specific scenarios that might pose challenges to the model, such as prompts with ambiguity or complex context and assess its performance under these conditions.

* Experiment with a wider range of LLM models during testing.

* Utilize GPT to formulate improved prompts and then compare the resulting outcomes to establish effectiveness.

* Create validation tests on the output (check returned keys, length).


3.

* There are cases where the format of the returned data may differ from JSON standards which can impact its reliability.

* Although the responses contain correct information, there are instances where they do not quite meet the level of excellence desired. Exploring various potential responses and making adjustments to hyperparameter values might bring about enhancements.

* Encountering challenges related to the broader context window (constrained by GPT-3's 4097 token limit) can result in exceptions.

* Attaining a cumulative word count of 5,000 words during camp generation presents a significant challenge.

* The conclusion of the registration process sometimes not ending properly.